## General Information

In this notebook, we are going to create a recommendation model which recommend the last radiant hero with the 5 heroes with the highest win rate. Using a dataset where the hero selection variables are already set, we run through to generate the team synergy and counter rates using the values calculated in the feature engineering


Using these values, we run it through the Neural Network model to predict the win rate for each hero lineup excluding those 9 heroes already selected. We then loop through the results to take the 5 heroes with the highest win rate to recommend to the player. The example that we tried was with a lineup as shown below with the recommended heroes generated. 


#### To simplify your navigation through this kernel:
    Main data exploration:
        1: Combine hero selection, synergy and Counter rate
        2: Predictive Modelling
        3: Model Evaluation Metrics
            -Classification Accuracy

### Change to your path

In [2]:
import os
os.chdir(r"C:\Users\chena\Desktop\dota-2-matches\code\all_csv_files")

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
matches = pd.read_csv('match.csv')
y = matches['radiant_win'].apply(lambda win: 1 if win else 0)
classes = ['Dire Win', 'Radiant Win']
X = pd.read_csv('final_dataset.csv')

###  Preparing dataset for hero selection + counter rate only

In [4]:
# First, we import python libraries which we will use for modelling and visualing the final model:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X=pd.DataFrame(scaler.transform(X),columns=X.columns)
X.head()

,radiant_Abaddon,radiant_Alchemist,radiant_Ancient Apparition,radiant_Anti-Mage,radiant_Axe,radiant_Bane,radiant_Batrider,radiant_Beastmaster,radiant_Bloodseeker,radiant_Bounty Hunter,...,dire_Visage,dire_Warlock,dire_Weaver,dire_Windranger,dire_Winter Wyvern,dire_Witch Doctor,dire_Wraith King,dire_Zeus,diff_counter,df_between_hero_synergy
0,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,-0.103506,-0.139616,-0.163512,-0.517351,-0.287529,-0.28145,-0.290373,-0.221033,0.893963,0.119532
1,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,-0.103506,-0.139616,-0.163512,-0.517351,-0.287529,-0.28145,-0.290373,4.524206,-0.062968,-1.558533
2,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,-0.103506,-0.139616,-0.163512,-0.517351,-0.287529,-0.28145,-0.290373,-0.221033,-1.308931,-0.779188
3,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,-0.103506,-0.139616,-0.163512,-0.517351,-0.287529,-0.28145,-0.290373,-0.221033,0.138878,-0.915655
4,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,-0.103506,-0.139616,-0.163512,1.932924,-0.287529,-0.28145,-0.290373,4.524206,0.848292,-0.321797


In [5]:
X1 = X.iloc[:,0:223]
X1.head()

,radiant_Abaddon,radiant_Alchemist,radiant_Ancient Apparition,radiant_Anti-Mage,radiant_Axe,radiant_Bane,radiant_Batrider,radiant_Beastmaster,radiant_Bloodseeker,radiant_Bounty Hunter,...,dire_Viper,dire_Visage,dire_Warlock,dire_Weaver,dire_Windranger,dire_Winter Wyvern,dire_Witch Doctor,dire_Wraith King,dire_Zeus,diff_counter
0,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,-0.194635,-0.103506,-0.139616,-0.163512,-0.517351,-0.287529,-0.28145,-0.290373,-0.221033,0.893963
1,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,-0.194635,-0.103506,-0.139616,-0.163512,-0.517351,-0.287529,-0.28145,-0.290373,4.524206,-0.062968
2,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,-0.194635,-0.103506,-0.139616,-0.163512,-0.517351,-0.287529,-0.28145,-0.290373,-0.221033,-1.308931
3,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,5.137826,-0.103506,-0.139616,-0.163512,-0.517351,-0.287529,-0.28145,-0.290373,-0.221033,0.138878
4,-0.186807,-0.331961,-0.269303,-0.319718,-0.219135,-0.162088,-0.100301,-0.114676,-0.175803,-0.268062,...,-0.194635,-0.103506,-0.139616,-0.163512,1.932924,-0.287529,-0.28145,-0.290373,4.524206,0.848292


## 2. Predictive Modelling (Neural Network)

In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from sklearn import model_selection
from tensorflow.keras import layers

Creating Neural Network Model (Based on previous experiences testing, having minimal layers with minimal neurons was more effective than increasing it constantly)

In [7]:
model = tf.keras.Sequential()  # input layer
model.add(layers.Dense(10, activation='relu', input_shape=(223,)))  
model.add(layers.Dense(30, activation=tf.nn.relu))      # one hidden layer
model.add(layers.Dense(30, activation=tf.nn.relu))      # one hidden layer
model.add(layers.Dense(30, activation=tf.nn.relu))      # one hidden layer
model.add(layers.Dense(30, activation=tf.nn.relu))      # one hidden layer

model.add(layers.Dense(1))

### 3. Evaluate model performance

In [8]:
model.compile(optimizer = tf.optimizers.Adam(),
             loss='squared_hinge',
             metrics=['accuracy'])

#### Training with Hero Selection + Counter only

In [9]:
#split data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X1, y, test_size=0.25, random_state=7)
#fit model
model.fit(x=X_train.values,y=y_train.values, epochs=10)
#accuracy on test set

Train on 37500 samples
Epoch 1/10
37500/37500 [==============================] - 4s 98us/sample - loss: 0.8467 - accuracy: 0.5642
Epoch 2/10
37500/37500 [==============================] - 2s 66us/sample - loss: 0.5882 - accuracy: 0.7022
Epoch 3/10
37500/37500 [==============================] - 2s 57us/sample - loss: 0.4848 - accuracy: 0.7672
Epoch 4/10
37500/37500 [==============================] - 3s 67us/sample - loss: 0.4543 - accuracy: 0.7854
Epoch 5/10
37500/37500 [==============================] - 3s 70us/sample - loss: 0.4397 - accuracy: 0.7914
Epoch 6/10
37500/37500 [==============================] - 2s 66us/sample - loss: 0.4297 - accuracy: 0.7977
Epoch 7/10
37500/37500 [==============================] - 2s 60us/sample - loss: 0.4224 - accuracy: 0.8025
Epoch 8/10
37500/37500 [==============================] - 2s 58us/sample - loss: 0.4147 - accuracy: 0.8072
Epoch 9/10
37500/37500 [==============================] - 2s 59us/sample - loss: 0.4073 - accuracy: 0.8118
Epoch 10/10
37

In [12]:
test_loss, test_acc = model.evaluate(X_test.values, y_test.values, verbose=0)
print ("Accuracy: " + str(test_acc))

Accuracy: 0.79552


### Top 5 hero recommender model based on the original lineup as listed in code below

In [13]:
rec = pd.read_csv("recommender.csv", index_col=0)
predictions = model.predict(rec.values)

In [14]:
original_lineup = ["radiant_Rubick", "radiant_Shadow Fiend", "radiant_Spectre", "radiant_Treant Protector", "dire_Abaddon", "dire_Alchemist", "dire_Earthshaker","dire_Ember Spirit","dire_Templar Assassin"]
def recommender_model(predictions): 
    min_val = 0
    index = 0
    recommended_index = []
    for i in range(5):
        for i in range(len(predictions)):
            for value in predictions[i]:
                if value >  min_val and i not in recommended_index:
                    index = i
                    min_val = value  
        recommended_index.append(index)
        min_val = 0
    return recommended_index

def dataset_lists(lineups):
    hero_lineup = []
    for i in range(len(lineups)):
        index = lineups[i]
        rec = pd.read_csv("recommender.csv",index_col=0)
        rec = rec.iloc[index: index+1]
        rec = rec.loc[:, (rec != 0).any(axis=0)]
        hero_lineup.append(list(rec.columns.values))
        
    return hero_lineup  
def get_top_5_heroes(lineups):
    recommended_heroes = []
    for i in range(len(lineups)):
        for j in range(len(lineups[i])):
            if lineups[i][j] not in original_lineup and lineups[i][j] != "diff":
                recommended_heroes.append(lineups[i][j])
    return recommended_heroes


In [15]:
lineups = recommender_model(predictions)

hero_lineups = dataset_lists(lineups)
top_5_heroes = get_top_5_heroes(hero_lineups)

top_5_heroes

['radiant_Weaver',
 'radiant_Omniknight',
 'radiant_Zeus',
 'radiant_Ogre Magi',
 'radiant_Spirit Breaker']